###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Time-domain viscoelastic equations of motion 

In the last lesson we proved that the Generalized Maxwell model (GMB) is able to describe a constant Q-spectrum, necessary for realistic SH wave propagation in viscoelastic media. Now, we only have to transform the frequency domain GMB stress-strain relation back to time-domain and assemble the forward problem in a form which can be solved by finite-differences.

## GMB stress-strain relation: from frequency to time domain

By combining a Maxwell Standard Linear Solid (SLS) in parallel with additional Maxwell bodies, we derived the **Generalized Maxwell Body (GMB)**

<img src="images/GMB.png" width="50%">

which can be described in the frequency domain by the stress-strain relation:

\begin{equation}
\tilde{\sigma} = \tilde{\mu}_{GMB} \tilde{\epsilon} \notag
\end{equation}

with the complex shear modulus:

\begin{equation}
\tilde{\mu}_{GMB} = \mu_0 + \delta \mu \sum_{l=1}^{L} \frac{i a_l \omega}{i \omega + \omega_l}. \notag
\end{equation}

where $\mu_0$ denotes the relaxed shear modulus, $\delta \mu$ the modulus defect, L the number of Maxwell bodies, $a_l, \omega_l$ weighting coefficients and relaxation frequencies of the l-th Maxwell body to achieve a constant Q-spectrum, while $\omega$ is  the circular frequency within the frequency range of the source wavelet.

For time-domain FD modelling, we obviously need to transform the complex modulus to time-domain by inverse Fourier transform leading to the relaxation function $\Psi(t)$

\begin{equation}
\Psi(t) = \frac{1}{2\pi}\int_{-\infty}^{\infty}\tilde{\mu}_{GMB} \exp(i\omega t) d\omega. \notag
\end{equation}

This leads to 

\begin{equation}
\Psi(t) = \biggl\{\mu_0 + \delta \mu \sum_{l=1}^L a_l \exp(-\omega_l t) \biggr\} \cdot H(t), \notag
\end{equation}

where $H(t)$ is the Heaviside step function. Using the definition of the modulus defect as the difference between the unrelaxed $\mu_u$ and relaxed shear modulus $\mu_0$:

\begin{equation}
\delta\mu = \mu_u - \mu_0\notag
\end{equation}

we can replace the relaxed by the unrelaxed modulus ([Emmerich & Korn 1987](https://library.seg.org/doi/10.1190/1.1442386), [Moczo & Kristek 2004](http://www.spice-rtn.org/library/publications/publicationbook.2007-05-08.3817028446.html)):

\begin{equation}
\Psi(t) = \biggl\{\mu_u - \delta \mu \sum_{l=1}^L a_l\biggl(1 - \exp(-\omega_l t)\biggr) \biggr\} \cdot H(t). \notag
\end{equation}

Before inserting the relaxation function into the viscoelastic stress-strain relation:

\begin{equation}
\sigma = \int_{-\infty}^{t}\dot{\Psi}(t-t')\epsilon(t')dt' = \dot{\Psi} * \epsilon\notag
\end{equation}

we have to take its first time derivative:

\begin{equation}
\dot{\Psi}(t) = - \delta \mu \sum_{l=1}^L a_l \omega_l \exp(-\omega_l t) \cdot H(t) + \biggl\{\mu_u - \delta \mu \sum_{l=1}^L a_l\biggl(1 - \exp(-\omega_l t)\biggr) \biggr\} \cdot \delta(t), \notag
\end{equation}

with the Delta function $\delta(t)$, where we used

\begin{equation}
\frac{\partial H(t)}{\partial t} = \delta(t)\notag
\end{equation}

Inserting the time-derivative of the relaxation function $\dot{\Psi}$ into the viscoelastic stress-strain relation yields:

\begin{align}
\sigma(t)=& -\int_{-\infty}^{t}\delta\mu \sum_{l=1}^L a_l \omega_l \exp(-\omega_l (t-t'))\cdot H(t-t')\; \epsilon(t') dt'\notag\\
&+ \int_{-\infty}^{t} \mu_0 \cdot \delta (t-t') \cdot \epsilon(t') dt'\notag\\
&-\int_{-\infty}^{t}\delta\mu \sum_{l=1}^L a_l \biggl(1 - \exp(-\omega_l (t-t'))\biggr)\cdot \delta(t-t')\; \epsilon(t') dt'\notag\\
\end{align}

Evaluating the Heaviside and Delta functions leads to 

\begin{equation}
\sigma(t) = \mu_u \cdot \epsilon(t) - \delta\mu \sum_{l=1}^L a_l \omega_l \int_{-\infty}^t\epsilon(t')\cdot \exp(-\omega_l (t-t')) dt'\notag
\end{equation}

The problem with this expression is the remaining time-convolution, which would be required to apply at each spatial grid point and at each time step during the FD time stepping, resulting in significant computational costs. To get rid of this convolution, we use a standard physicist trick: Step 1, we give the problem a new name, in this case we call:

\begin{equation}
\zeta_l(t) = \omega_l \int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt'\notag
\end{equation}

**memory variables** for the L Maxwell bodies $l=1,...,L$, leading to the stress-strain relation

\begin{equation}
\sigma(t) = \mu_u \cdot \epsilon(t) - \sum_{l=1}^L \delta\mu a_l \zeta_l(t),\notag
\end{equation}

where we do not have to apply the time convolution anymore. However, we need additional partial differential equations in order to estimate the memory variables $\zeta_l$. So in step 2 we take the first time derivative of the memory variables:

\begin{align}
\dot{\zeta}_l(t) &= \omega_l \frac{\partial}{\partial t}\int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt'\notag\\
&= \omega_l \biggl\{-\omega_l\int_{-\infty}^t \epsilon(t') \cdot \exp(-\omega_l (t-t')) dt' + \epsilon(t)\biggr\}\notag\\
&= \omega_l \biggl\{-\zeta_l(t) + \epsilon(t)\biggr\}\notag\\
\end{align}

With this trick, we could get rid of the time convolution, but at the price of L additional partial differential equations to estimate the memory variables, which have to be solved in the FD code:

\begin{equation}
\dot{\zeta}_l(t) = \omega_l \epsilon(t) - \omega_l \zeta_l(t)\notag
\end{equation}

To get the partial differential equations in the **stress-velocity** formulation, we only have to take the first time derivative, yielding:

\begin{align}
\dot{\sigma}(t) &= \mu_u \cdot \dot{\epsilon}(t) - \sum_{l=1}^L \delta\mu a_l \xi_l(t),
\notag\\
\dot{\xi}_l(t) &= \omega_l \dot{\epsilon}(t) - \omega_l \xi_l(t)\notag
\end{align}

with $\xi_l = \dot{\zeta_l}$. To simplify the parametrization, we introduce the dimensionless **anelastic coefficients** 

\begin{equation}
Y_l = a_l \frac{\delta \mu}{\mu_u}\notag
\end{equation}

leading to 

\begin{equation}
\dot{\sigma}(t) = \mu_u \cdot \dot{\epsilon}(t) - \sum_{l=1}^L \mu_u Y_l \xi_l(t).
\notag
\end{equation}

## Equations of motion for the viscoelastic 1D SH-problem

Finally, we combine the momentum equation with the stress-strain relation to the Equations of motion for the viscoelastic 1D SH-problem. After replacing the simplified notation from the last lesson:

\begin{align}
\sigma &\rightarrow \sigma_{yx}\notag\\
\epsilon &\rightarrow \epsilon_{yx}\notag\\
\mu_u &\rightarrow 2\mu_u\notag\\
\end{align}

we get the conservation of momentum:

\begin{equation}
\rho \dot{v}_y = \frac{\partial \sigma_{yx}}{\partial x} + f_y\notag
\end{equation}

and the viscoelastic stress-strain relation:

\begin{align}
\dot{\sigma_{yx}}(t) &= 2 \mu_u \dot{\epsilon_{yx}}(t) - \sum_{l=1}^L \mu_u Y_l \xi_l(t)\notag\\
\dot{\xi}_l(t) &= \omega_l \dot{\epsilon_{yx}}(t) - \omega_l \xi_l(t)\notag\\
\end{align}

with the unrelaxed shear modulus:

\begin{equation}
\mu_u = \frac{\mu_0}{1-\sum_{l=1}^L Y_l}\notag
\end{equation}

and 

\begin{equation}
\dot{\epsilon}_{yx} = \frac{1}{2} \frac{\partial v_y}{\partial x}\notag
\end{equation}

With these equations, the 1D viscoelastic SH problem is defined and can be solved by the staggered grid FD approach in the final TEW2 lesson.

## What we learned:

- Estimation of the time-domain relaxation function $\Psi(t)$ from the frequency-domain complex shear modulus of the GMB by inverse Fourier transform
- The time-domain viscoelastic stress-strain relation introduces a time convolution, required at each time-step and spatial subsurface point, which would significantly increase the computational costs of the FD modelling
- We can get rid of the time convolution by introducing memory variables, at the price of one additional partial differential equations for each Maxwell body
- Finally, we compiled the equations of motion for 1D wave propagation in viscoelastic SH media